In [ ]:
from fastai2.basics import *
from fastai2.vision.all import *
from faststyle import *

In [ ]:
source = untar_data(URLs.COCO_TINY)

In [ ]:
style_dir = Path('styles')
style_fns = L(style_dir/'abstract.jpg', style_dir/'starry_night.jpg')

In [ ]:
dblock = DataBlock(style_blocks, get_items=get_image_files, splitter=RandomSplitter(.1),
                   item_tfms=[Resize(128)],
                   batch_tfms=NormalizeX.from_stats(*coco_stats))

In [ ]:
dls = dblock.dataloaders(source, bs=4)

In [ ]:
get_feats = LayerFeats.from_feat_m(FeatModels.vgg19)

In [ ]:
source_tims = L(TensorImage.create(fn) for fn in style_fns)

In [ ]:
# source_tims = Normalize.from_stats(*imagenet_stats)(source_tims)

In [ ]:
stl_fts,cnt_fts = zip(*L(get_feats(o[None]) for o in source_tims))
stl_fts,cnt_fts = L(zip(*stl_fts)),L(zip(*cnt_fts))

In [ ]:
cbs = [SourceFeatsCallback(stl_fts, cnt_fts)]

In [ ]:
loss_func = FastStyleLoss2(get_feats)

In [ ]:
learn = style_learner(dls, style_fns, loss_func=loss_func, cbs=cbs)

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 1e-3)

In [ ]:
learn.show_results(max_n=1)